# Interactive Plots and Animations

So far we have shown you how to perform static plotting with Seaborn. But sometimes it is nice to go beyond that. We have several options how to do that:

- Interactive Plots: here the user can become active and zoom into the plot etc. We will do this via the Plotly library.
- GIFs: If you want to show how data evolves over time you can stack several plots over each other and create a gif. For this we will use the `gif` and the `imageio` library (two alternatives for the same goal).
- Animations: If we do not want the "laggy" effect of a GIF we can create fluent animations directly in matplotlib (although this is a bit more work).

## Interactive Plots with Plotly

For interactive plots there are two main libraries in Python: Bokeh and Plotly. Both are great tools, but we chose to go with Plotly just because there is more documentation and learning material available. Below you can find a short comparison table between the two taken from [this post](https://buggyprogrammer.com/bokeh-vs-plotly-which-one-is-better-in-2022/):

| Parameter                  | Bokeh                                                       | Plotly                                                                    |
|----------------------------|-------------------------------------------------------------|---------------------------------------------------------------------------|
| Dashboards                 | Bokeh serves dashboards using bokeh server                  | Plotly serves dashboard using the dash                                    |
| Plot features              | Plotting in Bokeh is a bit intense and has no 3D   graphing | Plotly code is simple and has 3D graph features                           |
| Ease of learning and   use | Styling graphs with bokeh is a tedious process              | Plotly code is simple and has 3D graph features                           |
| Data handling              | Bokeh handles data well and is fast                         | Plotly handles data as data frames and takes time to plot                 |
| Dashboard Interactions     | Dashboards with Bokeh server are dynamic and very fast      | Plotly dash is a bit static and takes time to load, hence it is very slow |

In [ ]:
# !pip install plotly
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
fig.show()

## Figure Data Structure 

Adapted from [here](https://plotly.com/python/figure-structure/)

Plotly works by converting your figures represented in code into JSON text and passing this to the JavaScript library Plotly.js. This however should not bother you at all; you only have to interact with Plotly Express, the high-level plotly module whose functions directly return fully-populated `plotly.graph_objects.Figure` objects that can converted into JSON by the program. You can look at the underlying data structure of such a figure obejct via `print(fig)`:

In [ ]:
print(fig)

## Dash - Build Web Application Dashboards

Plotly graphics can be easily converted to web dashboards and further extended via the [Dash library](https://dash.plotly.com/introduction). This is not the focus of today, but I will show you some code of it so that you can dig into it in case you are interested )[this video](https://www.youtube.com/watch?v=7m0Bq1EGPPg&t=0s) gives a good introduction as well).

If you know a bit of web development, Dash will seem very familiar to you since it is built on Plotly.js as well as React.js. 

A Dash web app can be described as **components** that are displayed on the webpage via the **layout** and which interact with each other via **callbacks**.

The code below is from [this GitHub repo](https://github.com/Coding-with-Adam/Dash-by-Plotly), check it out if you want to learn more about Dash!

In [ ]:
from dash import Dash, dcc               # pip install dash
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components

# Build your components
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
mytext = dcc.Markdown(children="# Hello World - let's build web apps in Python!")

# Customize your own Layout
app.layout = dbc.Container([mytext])

# Run app
if __name__=='__main__':
    app.run_server(port=8041)

Here we already had one component (our text) and the layout. But to make the webapp interactive, we need to add a callback:

In [ ]:
from dash import Dash, dcc, Output, Input  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components

# Build your components
app = Dash(__name__, external_stylesheets=[dbc.themes.SOLAR])
mytext = dcc.Markdown(children='')
myinput = dbc.Input(value="# Hello World - let's build web apps in Python!")

# Customize your own Layout
app.layout = dbc.Container([mytext, myinput])

# Callback allows components to interact
@app.callback(
    Output(mytext, component_property='children'),
    Input(myinput, component_property='value')
)
def update_title(user_input):  # function arguments come from the component property of the Input
    return user_input  # returned objects are assigned to the component property of the Output


# Run app
if __name__=='__main__':
    app.run_server(port=8042)

Now with our callbacks understood, we can build interactive graphs for our web app with `plotly.express` as shown before!

In [ ]:
from dash import Dash, dcc, Output, Input  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import plotly.express as px

# incorporate data into app
df = px.data.medals_long()

# Build your components
app = Dash(__name__, external_stylesheets=[dbc.themes.VAPOR])
mytitle = dcc.Markdown(children='# App that analyzes Olympic medals')
mygraph = dcc.Graph(figure={})
dropdown = dcc.Dropdown(options=['Bar Plot', 'Scatter Plot'],
                        value='Bar Plot',  # initial value displayed when page first loads
                        clearable=False)

# Customize your own Layout
app.layout = dbc.Container([mytitle, mygraph, dropdown])

# Callback allows components to interact
@app.callback(
    Output(mygraph, component_property='figure'),
    Input(dropdown, component_property='value')
)
def update_graph(user_input):  # function arguments come from the component property of the Input
    if user_input == 'Bar Plot':
        fig = px.bar(data_frame=df, x="nation", y="count", color="medal")

    elif user_input == 'Scatter Plot':
        fig = px.scatter(data_frame=df, x="count", y="nation", color="medal",
                         symbol="medal")

    return fig  # returned objects are assigned to the component property of the Output


# Run app
if __name__=='__main__':
    app.run_server(port=8043)

With a bit more work on the layout we can create amazing web apps such as this one here:

In [ ]:
from dash import Dash, dcc, Output, Input  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import plotly.express as px
import pandas as pd                        # pip install pandas

# incorporate data into app
# Source - https://www.cdc.gov/nchs/pressroom/stats_of_the_states.htm
df = pd.read_csv("https://raw.githubusercontent.com/Coding-with-Adam/Dash-by-Plotly/master/Good_to_Know/Dash2.0/social_capital.csv")
print(df.head())

# Build your components
app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])
mytitle = dcc.Markdown(children='')
mygraph = dcc.Graph(figure={})
dropdown = dcc.Dropdown(options=df.columns.values[2:],
                        value='Cesarean Delivery Rate',  # initial value displayed when page first loads
                        clearable=False)

# Customize your own Layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([mytitle], width=6)
    ], justify='center'),
    dbc.Row([
        dbc.Col([mygraph], width=12)
    ]),
    dbc.Row([
        dbc.Col([dropdown], width=6)
    ], justify='center'),

], fluid=True)

# Callback allows components to interact
@app.callback(
    Output(mygraph, 'figure'),
    Output(mytitle, 'children'),
    Input(dropdown, 'value')
)
def update_graph(column_name):  # function arguments come from the component property of the Input

    print(column_name)
    print(type(column_name))
    # https://plotly.com/python/choropleth-maps/
    fig = px.choropleth(data_frame=df,
                        locations='STATE',
                        locationmode="USA-states",
                        scope="usa",
                        height=600,
                        color=column_name,
                        animation_frame='YEAR')

    return fig, '# '+column_name  # returned objects are assigned to the component property of the Output


# Run app
if __name__=='__main__':
    app.run_server(debug=False, port=8044)

The creator of all of these Dask webapps I showed you made [great videos](https://www.youtube.com/@CharmingData) explaining these concepts and also released [a book about Dash](https://github.com/DashBookProject/Plotly-Dash) if you are interested in more detail!

## Animations

The example below shows you an example on how to set up an animation in Python via Matplotlib; the code is from [here](https://colab.research.google.com/github/jckantor/CBE30338/blob/master/docs/A.03-Animation-in-Jupyter-Notebooks.ipynb#scrollTo=mj8OrIcBnyqI) where they describe in more detail what each step is doing.

You should be able to run the linked Colab above as it is. To run the animation your local Jupyter notebook, you will need to install the mediatool `ffmpeg` (for Mac for example via the commands `brew install yasm` and `brew install ffmpeg`).

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# create a figure and axes
fig = plt.figure(figsize=(12,5))
ax1 = plt.subplot(1,2,1)   
ax2 = plt.subplot(1,2,2)

# set up the subplots as needed
ax1.set_xlim(( 0, 2))            
ax1.set_ylim((-2, 2))
ax1.set_xlabel('Time')
ax1.set_ylabel('Magnitude')

ax2.set_xlim((-2,2))
ax2.set_ylim((-2,2))
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_title('Phase Plane')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
txt_title = ax1.set_title('')
line1, = ax1.plot([], [], 'b', lw=2)     # ax.plot returns a list of 2D line objects
line2, = ax1.plot([], [], 'r', lw=2)
pt1, = ax2.plot([], [], 'g.', ms=20)
line3, = ax2.plot([], [], 'y', lw=2)

ax1.legend(['sin','cos']);

In [ ]:
# animation function. This is called sequentially
def drawframe(n):
    x = np.linspace(0, 2, 1000)
    y1 = np.sin(2 * np.pi * (x - 0.01 * n))
    y2 = np.cos(2 * np.pi * (x - 0.01 * n))
    line1.set_data(x, y1)
    line2.set_data(x, y2)
    line3.set_data(y1[0:50],y2[0:50])
    pt1.set_data(y1[0],y2[0])
    txt_title.set_text('Frame = {0:4d}'.format(n))
    return (line1,line2)

In [ ]:
from matplotlib import animation

# blit=True re-draws only the parts that have changed.
anim = animation.FuncAnimation(fig, drawframe, frames=100, interval=20, blit=True)

In [ ]:
import matplotlib.animation as manimation; manimation.writers.list()

In [ ]:
from IPython.display import HTML
HTML(anim.to_html5_video())

## GIFs

There are several libraries available to create simple GIFs from matplotlib, such as `imageio` or `gif`.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio

Let's try a few short examples from the blog post [here](https://towardsdatascience.com/basics-of-gifs-with-pythons-matplotlib-54dd544b6f30):

In [ ]:
y = np.random.randint(30, 40, size=(40))

filenames = []
for i in y:
    # plot the line chart
    plt.plot(y[:i])
    plt.ylim(20,50)
    
    # create file name and append it to a list
    filename = f'{i}.png'
    filenames.append(filename)
    
    # save frame
    plt.savefig(filename)
    plt.close()

# build gif
with imageio.get_writer('mygif.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)

Same for a bar plot:

In [ ]:
n_frames = 10
bg_color = '#95A4AD'
bar_color = '#283F4E'
gif_name = 'bars'

x = [1, 2, 3, 4, 5]
coordinates_lists = [[0, 0, 0, 0, 0],
                     [10, 30, 60, 30, 10],
                     [70, 40, 20, 40, 70],
                     [10, 20, 30, 40, 50],
                     [50, 40, 30, 20, 10],
                     [75, 0, 75, 0, 75],
                     [0, 0, 0, 0, 0]]
print('Creating charts\n')
filenames = []
for index in np.arange(0, len(coordinates_lists)-1):
    y = coordinates_lists[index]
    y1 = coordinates_lists[index+1]    
    
    y_path = np.array(y1) - np.array(y)    
    
    for i in np.arange(0, n_frames + 1):
        y_temp = (y + (y_path / n_frames) * i)        
        
        # plot
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.set_facecolor(bg_color)       
        
        plt.bar(x, y_temp, width=0.4, color=bar_color)
        plt.ylim(0,80)  
        
        # remove spines
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)        
        
        # grid
        ax.set_axisbelow(True)
        ax.yaxis.grid(color='gray', linestyle='dashed', alpha=0.7)        
        
        # build file name and append to list of file names
        filename = f'frame_{index}_{i}.png'
        filenames.append(filename)
        
        # last frame of each viz stays longer
        if (i == n_frames):
            for i in range(5):
                filenames.append(filename)        
                
        # save img
        plt.savefig(filename, dpi=96, facecolor=bg_color)
        plt.close()
print('Charts saved\n')

# Build GIF
print('Creating gif\n')
with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
print('Gif saved\n')
print('Removing Images\n')
# Remove files
for filename in set(filenames):
    os.remove(filename)
print('DONE')

And finally with a scatter plot:

In [ ]:
coordinates_lists = [[[0],[0]],
                     [[100,200,300],[100,200,300]],
                     [[400,500,600],[400,500,600]],
                     [[400,500,600,400,500,600],[400,500,600,600, 500,400]],
                     [[500],[500]],
                     [[0],[0]]]

gif_name = 'movie' 
n_frames=10

bg_color='#95A4AD'
marker_color='#283F4E' 
marker_size = 25

print('building plots\n')
filenames = []
for index in np.arange(0, len(coordinates_lists)-1):
    # get current and next coordinates
    x = coordinates_lists[index][0]
    y = coordinates_lists[index][1]    
    
    x1 = coordinates_lists[index+1][0]
    y1 = coordinates_lists[index+1][1]    
    
    # Check if sizes match
    while len(x) < len(x1):
        diff = len(x1) - len(x)
        x = x + x[:diff]
        y = y + y[:diff]    

    while len(x1) < len(x):
        diff = len(x) - len(x1)
        x1 = x1 + x1[:diff]
        y1 = y1 + y1[:diff]    
        
    # calculate paths
    x_path = np.array(x1) - np.array(x)
    y_path = np.array(y1) - np.array(y)    
    
    for i in np.arange(0, n_frames + 1):                
        # calculate current position
        x_temp = (x + (x_path / n_frames) * i)
        y_temp = (y + (y_path / n_frames) * i)        
        
        # plot
        fig, ax = plt.subplots(figsize=(6, 6), subplot_kw = dict(aspect="equal"))
        ax.set_facecolor(bg_color)
            
        plt.scatter(x_temp, y_temp, c=marker_color, s = marker_size)        
        
        plt.xlim(0,1000)
        plt.ylim(0,1000)        
        
        # remove spines
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)        
        
        # grid
        ax.set_axisbelow(True)
        ax.yaxis.grid(color='gray', linestyle='dashed', alpha=0.7)
        ax.xaxis.grid(color='gray', linestyle='dashed', alpha=0.7)        
        
        # build file name and append to list of file names
        filename = f'frame_{index}_{i}.png'
        filenames.append(filename)        
        
        if (i == n_frames):
            for i in range(5):
                filenames.append(filename)        
                
        # save img
        plt.savefig(filename, dpi=96, facecolor=bg_color)
        plt.close()
        
# Build GIF
print('creating gif\n')
with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

print('gif complete\n')
print('Removing Images\n')
# Remove files
for filename in set(filenames):
    os.remove(filename)
print('done')

The guy creating these animations above even made [a library for it](https://github.com/Thiagobc23/Scatter-Letters), so you can see how quickly a side project become something bigger!

You can do similar stuff with the `gif` library (see [this post](https://towardsdatascience.com/a-simple-way-to-turn-your-plots-into-gifs-in-python-f6ea4435ed3c) for the examples I use below):

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

import matplotlib.pyplot as plt
import gif

# settings
plt.style.use("seaborn")
gif.options.matplotlib["dpi"] = 300

# !pip install yfinance
# !pip install gif

In [ ]:
#download stock data from Tesla
df = yf.download("TSLA", 
                 start="2019-01-01", 
                 end="2021-12-31")

df.head()

In [ ]:
#get monthly instead of daily data
tsla_df = df[["Adj Close"]].resample("M").last()

Helper function to separately display each frame of the animation

In [ ]:
@gif.frame
def helper_plot_1(df, i):
    df = df.copy()
    df.iloc[i:] = np.nan #mask data after current step so that we display entire x axis from start
    ax = df.plot(title="Tesla's stock price", legend=False, style="o--")
    ax.set_xlabel("")
    ax.set_ylabel("Price ($)")
    #with matplotlib the function does not return anything, with plotly it would

In [ ]:
frames = []
for i in range(1, len(tsla_df)):
    frames.append(helper_plot_1(tsla_df, i))

In [ ]:
gif.save(frames, "tesla_stock_price.gif", 
         duration=15)

Do the same think for multiple time series:

In [ ]:
df = yf.download(["TSLA", "TWTR", "AMZN", "AAPL"], 
                 start="2019-01-01", 
                 end="2021-12-31")

df = df[["Adj Close"]].droplevel(0, axis=1).resample("M").last()
df = df.div(df.iloc[0])
df.head()

In [ ]:
@gif.frame
def helper_plot_2(df, i):
    
    df = df.copy()
    df.iloc[i:] = np.nan
    
    ax = df.plot(title="Selected stocks' change of value")
    ax.set_xlabel("")
    
    # move the legend below the plot
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                     box.width, box.height * 0.9])
    ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1),
              fancybox=True, shadow=True, ncol=5)

More info on how to work with the legend [here](https://stackoverflow.com/questions/4700614/how-to-put-the-legend-out-of-the-plot).

In [ ]:
frames = []
for i in range(1, len(df)):
    frames.append(helper_plot_2(df, i))

In [ ]:
gif.save(frames, "tech_stock_prices.gif", 
         duration=15)